# Blackbody radiation formula

## Planck's Law

$$B_\lambda (T) = \frac{2hc^2}{\lambda^5 e^{\frac{hc}{\lambda k_B T}}-1}$$

Where $B$ is the amount of energy emitted at a given wavelength, $h$ is Planck's constant, $k_B$ is the Boltzmann constant, and $c$ is the speed of light.

$B_\lambda$ has units of $W\cdot sr^{-1}\cdot m^{-3}$.

Note that:

$k_B = 1.480 649 \times 10^{-23} J/K$

$h = 6.626 070 15 \times 10^{-34} J\cdot s$

$c = 299 792 458 m/s$

## Wavelength of maximum emission (Wien's Law)

We can find the wavelength where the emission peaks by using:

$$\lambda = \frac{2.898 \times 10^{-3}}{T}$$

In [1]:
# sphinx_gallery_thumnail_number = 3
%matplotlib inline
import IPython
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as sc
import ipywidgets as widgets
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from matplotlib.patches import Rectangle
import pandas as pd

In [2]:
# For our x-axis. Create a set of numbers from 1x10^-8 to 2x10^-6 in increments of 1x10^-9
# Note that starting at 1x10^-9, (1 nm), where I'd like, gets an overflow error.
lamda = np.arange(1e-8, 2e-6, 5e-9)
# A couple constants. Boltzman's, Planck's, and the speed of light.
B = sc.Boltzmann
h = sc.h
c = sc.c

# temp_to_rgb is based on a file that contains a bunch of rgb values for different temperatures.
# Data from:
# Mitchell Charity <mcharity@lcs.mit.edu>
# http://www.vendian.org/mncharity/dir3/blackbody/
# Load the data into a pandas dataframe.
temp_to_rgb = pd.read_pickle('temp_to_rgb.data')

# Function to calculate our curve based on a temperature and wavelength.
# Note that this is used by the plotting software. T is set by the slider.
# Wav is our x-axis values.
def planck(wav, T):
    numerator = 2*h*c**2
    exponent = np.exp(h*c/(wav*B*T))
    denominator = wav**5 * (exponent - 1.0)
    return numerator/denominator

# Calculate the peak wavelength for the curve using Wien's Law.
def maxlamda(T):
    return (2.898*1e-3)/T

# Set the style of our plot. A dark background seemed appropriate for the topic.
plt.style.use('dark_background')
# Make it a larger plot than normal. Default option is pretty small.
plt.rcParams['figure.figsize'] = [16,6]

# Annoying thing we have to do to get our graph to update. I want to find a better way
# to do this at some point. All of this stuff in here makes the whole thing slow.
# This function is called whenever the slider is updated.
def g(T):
    # Generate the actual blackbody curve.
    bbr_curve = plt.plot(lamda*1e9, planck(lamda, T), label=f'{T} K')
    # Get the axes for our plot.
    ax = plt.gca()
    # Since our x and y axes can change, get our max values. We'll need them later. Toss the min.
    _, curymax = ax.get_ylim()
    _, curxmax = ax.get_xlim()
    
    # Now that we have our curve, calculate our peak value. Draw a line there and add it to the legend.
    max_lamda = maxlamda(T)
    plt.axvline(max_lamda*1e9, color='r', label=f'Max wavelength: {max_lamda*1e9:3.1f} nm')
    
    # Style our grid, including titles and axes labels.
    plt.grid(visible=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
    plt.xlabel('$\lambda$ (Wavelength) ($10^{-9}$ m)')
    plt.ylabel('Energy (W$\cdot$ sr$^{-1}\cdot$ m$^{-3}$)')
    plt.title('Blackbody Radiation', fontsize=14)
    
    # Show the appropriate colors. This should be able to be done through a Bbox, but I don't
    # have that working yet. These create small boxes. The first two are the min and max values.
    # They are based on definitions of the color ranges.
    plt.axvspan(700, curxmax, facecolor='#ff0000', alpha=0.05) #Infrared
    plt.axvspan(635, 700, facecolor='#ff0000', alpha=0.5) # Red
    plt.axvspan(590, 635, facecolor='#ff7f00', alpha=0.5) # Orange
    plt.axvspan(560, 590, facecolor='#ffff00', alpha=0.5) # Yellow
    plt.axvspan(520, 560, facecolor='#00ff00', alpha=0.5) # Green
    plt.axvspan(490, 520, facecolor='#00ffff', alpha=0.5) # Cyan
    plt.axvspan(450, 490, facecolor='#0000ff', alpha=0.5) # Blue
    plt.axvspan(400, 450, facecolor='#7F00FF', alpha=0.5) # Violet
    plt.axvspan(10, 400, facecolor='#7F00FF', alpha=0.05) # UV
    
    # Put labels in for the IR and UV sections.
    ax.annotate('Infrared', xy=(1000, curymax/2), fontsize=12)
    ax.annotate('Ultraviolet', xy=(150, curymax/2), fontsize=12)
    
    # Get the rgb color that goes with our current temperature from the pandas dataframe.
    # This says: Find the right temperature. Get the value under rgb. Pull out just the piece of data.
    colorval = temp_to_rgb.loc[temp_to_rgb["Temperature"] == T, "rgb"].iloc[0]
    # Create a rectangle to put our color in and label it.
    showtemp = Rectangle((1750, curymax/2), 300, 0.2*curymax, facecolor=colorval)
    ax.annotate("Object Color", xy=(1800, curymax/2 + 0.22*curymax), fontsize=14)
    ax.add_patch(showtemp)
    
    # Generate our legend.
    plt.legend()
    # Finally, show the actual plot.
    plt.show()

slider_style = {'description_width': 'initial'}
slider_layout = widgets.Layout(width='400px')

# Here's our widget. It calls our update function, g. We're using a slider with some simple defaults.
interactive_plot = widgets.interactive(g, T=widgets.IntSlider(
    min=2100, 
    max=20000, 
    value=3000, 
    step=100,
    description="Temperature (K)",
    style=slider_style,
    layout=slider_layout,
))
# Actually show the slider.
interactive_plot

interactive(children=(IntSlider(value=3000, description='Temperature (K)', layout=Layout(width='400px'), max=2…

# What am I looking at?

Every object that has a temperature gives off light. The light given off depends on the temperature of the object. Think of a piece of iron that is heated up. Initially, at room temperature, it doesn't look like it's giving off any light. With the right tools, though, you can see that it's giving off infrared light. As you heat it up, however, the iron will start to glow a dull red and then move more toward a white color. Really, everything does this, we just don't have a way to see it with our eyes alone.

This graph shows you the amount of light given off at different wavelengths. The x-axis shows the different wavelengths (visible light areas are approximated on the graph based on common descriptions of color ranges) of light given off. The y-axis shows how much light (well, energy, technically) is given off at those wavelengths. Notice that the scale for the y-axis changes. This is because there's a huge difference in the amount of light given off by a cooler object compared to one that is much hotter. If we used one scale for all of them it would make it very difficult to see.

In addition, the graph highlights the peak wavelength given off by something and the approximate color of the object.

Data for temperature to color conversion taken from the work of [Mitchell Charity](mcharity@lcs.mit.edu) at <http://www.vendian.org/mncharity/dir3/blackbody/>.

Built by D. Vetter, 2022. <dvetter@protonmail.com> using [Voila](https://github.com/voila-dashboards/voila) and [JupyterLab](https://jupyter.org/).

[![DigitalOcean Referral Badge](https://web-platforms.sfo2.cdn.digitaloceanspaces.com/WWW/Badge%201.svg)](https://www.digitalocean.com/?refcode=ec55721203e6&utm_campaign=Referral_Invite&utm_medium=Referral_Program&utm_source=badge)